# Data Validation with Great Expectations
**By Jayden Nyamiaka**

In this notebook, we will become familiar with Python data validation techniques using the Great Expectations module. We will work with a spreadsheet containing 673 data scientist job postings scraped from Glassdoor and validate the data according to our expectations of what criteria "quality" data should meet. Often data validation requires pre-processing the data (creating additional variables, changing/filtering specific values, etc) before working with the data directly. This exercise will expose to data preprocessing and validation.

According to our industry knowledge, the following expectations should hold:
1) Salary Estimate are between 50k and 400k
2) Salary Estimate are listed as {A } – {B } where A is strictly smaller than B.
3) Rating is between 0 and 5
4) Location of the company is either remote or in the United States
5) Size variable contains the pattern {integer} – to – {integer} employees
6) Founded to be later than the year 1800
7) All firms founded in 2019 should not have Revenue (i.e. Unknown / Non-Applicable)

We will validate the data using these expectations.

In [25]:
# Imports
import pandas as pd
import great_expectations as ge

In [26]:
## Import data
filename = "glassdoor_data_scientist_postings.xlsx"
df = pd.read_excel(filename)

## View sample of the data
print(df.head())

   index          Job Title               Salary Estimate  \
0      0  Sr Data Scientist  $137K-$171K (Glassdoor est.)   
1      1     Data Scientist  $137K-$171K (Glassdoor est.)   
2      2     Data Scientist  $137K-$171K (Glassdoor est.)   
3      3     Data Scientist  $137K-$171K (Glassdoor est.)   
4      4     Data Scientist  $137K-$171K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Description\n\nThe Senior Data Scientist is re...     3.1   
1  Secure our Nation, Ignite your Future\n\nJoin ...     4.2   
2  Overview\n\n\nAnalysis Group is one of the lar...     3.8   
3  JOB DESCRIPTION:\n\nDo you have a passion for ...     3.5   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

              Company Name       Location            Headquarters  \
0         Healthfirst\n3.1   New York, NY            New York, NY   
1             ManTech\n4.2  Chantilly, VA             Herndon, VA   
2      Analysis Group\n3.8     Boston, MA

In [27]:
# Data Preprocessing
df.replace(-1, pd.NA, inplace=True)

'''def extract_int_founded_year(year_str):
    if isinstance(year_str, str):
        print(year_str)
        return year_str.strip().replace(',', '')
    return year_str'''
def extract_min_salary(estimate):
    return int(estimate[1:estimate.find('K')])
def extract_max_salary(estimate):
    return int(estimate[estimate.find('K')+3:estimate.rfind('K')])
#df["Founded Int"] = df["Founded"].apply(extract_int_founded_year)
df['Min Salary'] = df["Salary Estimate"].apply(extract_min_salary)
df['Max Salary'] = df["Salary Estimate"].apply(extract_max_salary)
df['Salary Range'] = df['Max Salary'] - df['Min Salary']


In [28]:
# Data Validation

# Convert Pandas DataFrame to Great Expectations DataFrame
ge_df = ge.from_pandas(df)

# Expectation 1
ge_df.expect_column_values_to_be_between("Min Salary", min_value=50, max_value=400)
ge_df.expect_column_values_to_be_between("Max Salary", min_value=50, max_value=400)
# Expectation 2
ge_df.expect_column_values_to_be_between("Salary Range", min_value=0, max_value=None, strict_min=True)
# Expectation 3
ge_df.expect_column_values_to_be_between("Rating", min_value=0, max_value=5)
# Expectation 4
valid_places = ["Remote", "United States", "USA", "US",
    "Alabama", "AL", "Kentucky", "KY", "Ohio", "OH",
    "Alaska", "AK", "Louisiana", "LA", "Oklahoma", "OK",
    "Arizona", "AZ", "Maine", "ME", "Oregon", "OR",
    "Arkansas", "AR", "Maryland", "MD", "Pennsylvania", "PA",
    "American Samoa", "AS", "Massachusetts", "MA", "Puerto Rico", "PR",
    "California", "CA", "Michigan", "MI", "Rhode Island", "RI",
    "Colorado", "CO", "Minnesota", "MN", "South Carolina", "SC",
    "Connecticut", "CT", "Mississippi", "MS", "South Dakota", "SD",
    "Delaware", "DE", "Missouri", "MO", "Tennessee", "TN",
    "District of Columbia", "DC", "Montana", "MT", "Texas", "TX",
    "Florida", "FL", "Nebraska", "NE", "Trust Territories", "TT",
    "Georgia", "GA", "Nevada", "NV", "Utah", "UT",
    "Guam", "GU", "New Hampshire", "NH", "Vermont", "VT",
    "Hawaii", "HI", "New Jersey", "NJ", "Virginia", "VA",
    "Idaho", "ID", "New Mexico", "NM", "Virgin Islands", "VI",
    "Illinois", "IL", "New York", "NY", "Washington", "WA",
    "Indiana", "IN", "North Carolina", "NC", "West Virginia", "WV",
    "Iowa", "IA", "North Dakota", "ND", "Wisconsin", "WI",
    "Kansas", "KS", "Northern Mariana Islands", "MP", "Wyoming", "WY"
]
places_regex = rf"\b({"|".join(valid_places)})\b"
ge_df.expect_column_values_to_match_regex("Location", places_regex)
# Expectation 5
ge_df.expect_column_values_to_match_regex("Size", r"[0-9]+ to [0-9]+ employees")
# Expectation 6
ge_df.expect_column_values_to_be_between("Founded", min_value=1800, max_value=None, strict_min=True)
# Expectation 7
ge_df.expect_column_values_to_be_in_set("Revenue", 
    value_set=['Unknown / Non-Applicable'], 
    condition_parser="pandas",
    row_condition="Founded == 2019")

# Validate the data
results = ge_df.validate(result_format="COMPLETE")
print(results)

{
  "success": false,
  "results": [
    {
      "success": false,
      "expectation_config": {
        "expectation_type": "expect_column_values_to_be_between",
        "kwargs": {
          "column": "Min Salary",
          "min_value": 50,
          "max_value": 400,
          "result_format": "COMPLETE"
        },
        "meta": {}
      },
      "result": {
        "element_count": 672,
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_count": 21,
        "unexpected_percent": 3.125,
        "unexpected_percent_total": 3.125,
        "unexpected_percent_nonmissing": 3.125,
        "partial_unexpected_list": [
          990,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31,
          31
        ],
        "partial_unexpected_index_list": [
          13

In [31]:
# Extract set of invalid indices from the returned JSON
invalid_indices = set()
for result in results["results"]:
    invalid_indices.update(result["result"]["unexpected_index_list"])
invalid_indices = sorted(invalid_indices)
print("Number of Invalid Indices: " + str(len(invalid_indices)))
print("Invalid Indices: ")
print(*invalid_indices, sep="\n")

Number of Invalid Indices: 121
Invalid Indices: 
6
31
41
45
51
60
73
80
89
90
97
108
109
138
150
155
159
164
168
169
181
187
189
192
193
194
204
215
235
237
245
246
252
256
258
261
268
272
274
277
279
282
285
286
287
292
295
299
300
308
321
334
336
347
354
365
370
372
383
395
401
409
417
420
424
430
434
442
444
448
449
452
455
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
493
508
509
513
524
535
536
548
568
570
578
581
589
591
594
595
599
607
611
614
615
617
631
644
645
651
659
661
